In [ ]:
# !pip install boto3 easydict pymysql smart_open

In [4]:
import boto3
boto3.__version__

'1.26.72'

In [72]:
from easydict import EasyDict

settings = EasyDict()

settings.AWS_ACCESS_KEY_ID = "YOUR_ACCESS_KEY_ID" # AWS 자격증명 ID
settings.AWS_SECRET_ACCESS_KEY = "YOUR_SECRECT_ACCESS_KEY" # AWS 자격증명 PW
settings.REGION_NAME = "ap-northeast-2" # AWS 지역
settings.AWS_ACCOUNT_ID = "YOUR_ACCOUNT_ID" # AWS 계정 ID

settings

{'AWS_ACCESS_KEY_ID': 'YOUR_ACCESS_KEY_ID',
 'AWS_SECRET_ACCESS_KEY': 'YOUR_SECRECT_ACCESS_KEY',
 'REGION_NAME': 'ap-northeast-2',
 'AWS_ACCOUNT_ID': 'YOUR_ACCOUNT_ID'}

In [20]:
import boto3

s3_client = boto3.client(
    service_name="s3",
    aws_access_key_id=settings.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=settings.AWS_SECRET_ACCESS_KEY,
    region_name=settings.REGION_NAME,
)

s3_client

In [21]:
settings.AWS_BUCKET_NAME = "genia-bucket"

obj = s3_client.get_object(Bucket=settings.AWS_BUCKET_NAME, Key="raw/abalone.csv")
obj

{'ResponseMetadata': {'RequestId': 'PM1P100QRCX490NK',
  'HostId': 'pE4K8h/KkKUbChLv+7sHvO+E72H628TkcBXBIGA+bmq4zQy3FV5tp1LM9TcQWHEyiEOef/79YW8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pE4K8h/KkKUbChLv+7sHvO+E72H628TkcBXBIGA+bmq4zQy3FV5tp1LM9TcQWHEyiEOef/79YW8=',
   'x-amz-request-id': 'PM1P100QRCX490NK',
   'date': 'Thu, 16 Feb 2023 05:13:48 GMT',
   'last-modified': 'Thu, 16 Feb 2023 04:54:53 GMT',
   'etag': '"04e42306f2df277c7e660afeceb1f829"',
   'x-amz-server-side-encryption': 'AES256',
   'accept-ranges': 'bytes',
   'content-type': 'text/csv',
   'server': 'AmazonS3',
   'content-length': '216903'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2023, 2, 16, 4, 54, 53, tzinfo=tzutc()),
 'ContentLength': 216903,
 'ETag': '"04e42306f2df277c7e660afeceb1f829"',
 'ContentType': 'text/csv',
 'ServerSideEncryption': 'AES256',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7fb278650370>}

In [24]:
import pandas as pd

df = pd.read_csv(obj["Body"], encoding="utf-8")
df

,Type,LongestShell,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [39]:
# csv 파일 형태로 저장
df.to_csv("./teacher.csv", encoding="utf-8", index=False)

In [47]:
# Parquet 형태로 저장
df.to_parquet(
    "./teacher.parquet",
    engine="pyarrow",
    compression="gzip",
    index=False
)

In [ ]:
# put_object로 업로드: 지원 타입: [Bytes, file path(str)]
s3_client.put_object(
    Body="./teacher.parquet",
    Bucket=settings.AWS_BUCKET_NAME,
    Key="upload-test/teacher.parquet"
)

In [48]:
# upload_file로 업로드: 지원타입: [file path(str)]
response = s3_client.upload_file(
    Filename='./teacher.parquet',
    Bucket=settings.AWS_BUCKET_NAME,
    Key="upload-test/teacher.parquet",
)

response

In [54]:
from smart_open import open as s_open

# smart_open library로 업로드: 메모리 상의 데이터를 로컬 파일 저장 없이 바로 업로드
file_name = f"s3://{settings.AWS_BUCKET_NAME}/glue-test/teacher.parquet"

with s_open(file_name, "wb", transport_params=dict(client=s3_client)) as out_file:
    df.to_parquet(out_file, engine="pyarrow", compression="gzip", index=False)

In [5]:
import pymysql
from easydict import EasyDict

settings = EasyDict()
settings.DB_HOST = "YOUR_DB_HOST"
settings.DB_USER = "YOUR_DB_USER"
settings.DB_PASSWORD = "YOUR_DB_PASSWORD"
settings.DB_NAME = "YOUR_DB_NAME"
settings.DB_PORT = 3306

db = pymysql.connect(
    host=settings.DB_HOST,
    port=settings.DB_PORT,
    user=settings.DB_USER,
    passwd=settings.DB_PASSWORD,
    db=settings.DB_NAME,
)

cursor = db.cursor()

In [8]:
import pandas as pd

def get_query_to_df(sql: str):
    cursor.execute(sql)
    data = cursor.fetchall()
    columns = [elem[0] for elem in cursor.description]
    return pd.DataFrame(data, columns=columns)

In [9]:
sql = "SELECT * FROM hotel LIMIT 10;"
df = get_query_to_df(sql)
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,None,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,None,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,None,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,None,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,None,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [10]:
sql = "SELECT * FROM hotel WHERE arrival_date_year=2015 && arrival_date_month='July'"
df = get_query_to_df(sql)
df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2771,City Hotel,0,1,2015,July,31,30,0,1,1,...,No Deposit,NaN,NaN,0,Transient,0.00,0,1,Check-Out,2015-07-31
2772,City Hotel,0,210,2015,July,31,30,0,1,2,...,No Deposit,9.0,NaN,0,Transient-Party,76.50,0,0,Check-Out,2015-07-31
2773,City Hotel,0,1,2015,July,31,30,0,1,1,...,No Deposit,NaN,NaN,0,Transient,0.00,0,1,Check-Out,2015-07-31
2774,City Hotel,0,209,2015,July,31,29,1,4,1,...,No Deposit,7.0,NaN,0,Transient,55.96,0,0,Check-Out,2015-08-03
